Command to run Real-ESRGAN:

python inference_realesrgan.py -n RealESRGAN_x4plus -i inputs --outscale 4 --face_enhance

In [10]:
# This block is for generating a sample of our dataset to run tests on (since it takes too long to run tests on the entire set)

import glob
import cv2
import os
import random

picks = random.choices((glob.glob('./Real-ESRGAN/datasets/ln_covers/*')), k=100)
for path in picks:
    img_name = os.path.basename(path)
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        continue
    img_LR = cv2.resize(img, (img.shape[1] // 4, img.shape[0] // 4), interpolation=cv2.INTER_AREA)
    cv2.imwrite('./Real-ESRGAN/dataset_sample/LR/' + img_name, img_LR)
    cv2.imwrite('./Real-ESRGAN/dataset_sample/Orig/' + img_name, img)

./Real-ESRGAN/dataset_sample/LR/Zatsuki-Make-Me-a-Star.jpg
./Real-ESRGAN/dataset_sample/LR/Irina-The-Vampire-Cosmonaut-v06.jpg
./Real-ESRGAN/dataset_sample/LR/The-Worlds-Fastest-Level-Up-Vol.-_3.jpg
./Real-ESRGAN/dataset_sample/LR/MANGACBZ-Goblin-Slayer.jpg
./Real-ESRGAN/dataset_sample/LR/From-the-Red-Fog.jpg
./Real-ESRGAN/dataset_sample/LR/MUSHOKU-VOL-13.jpg
./Real-ESRGAN/dataset_sample/LR/4ba159d9-fa62-45d3-a389-267d97221e02.jpg
./Real-ESRGAN/dataset_sample/LR/The-Most-Heretical-Last-Boss-Queen-From-Villainess-to-Savior-volume-1.jpg
./Real-ESRGAN/dataset_sample/LR/the-unwanted-undead-adventurer-volume-11.jpg
./Real-ESRGAN/dataset_sample/LR/Combatants-Will-Be-Dispatched-Volume-7.jpg
./Real-ESRGAN/dataset_sample/LR/enough-with-this-slow-life-i-was-reincarnated-as-a-high-elf-and-now-i-m-bored-volume-2.jpg
./Real-ESRGAN/dataset_sample/LR/Tsugumomo.jpg
./Real-ESRGAN/dataset_sample/LR/TBATEVOL-8.5.jpg
./Real-ESRGAN/dataset_sample/LR/41321aaf-a29a-41c6-a6bc-714ec8a52950.jpg
./Real-ESRGAN/da

In [17]:
from math import ceil
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse

running_ssim = 0
running_psnr = 0
running_mse = 0
with open('results.txt', 'w') as metrics:
    for res, orig in zip(os.listdir('./Real-ESRGAN/results/'), os.listdir('./Real-ESRGAN/dataset_sample/Orig/')):
        output = cv2.imread('./Real-ESRGAN/results/' + res, cv2.IMREAD_COLOR)
        img = cv2.imread('./Real-ESRGAN/dataset_sample/Orig/' + orig, cv2.IMREAD_COLOR)
        diffs = ((img.shape[0] - output.shape[0])//2, img.shape[0]-int(ceil((img.shape[0] - output.shape[0])/2)),
        (img.shape[1] - output.shape[1])//2, img.shape[1]-int(ceil((img.shape[1] - output.shape[1])/2)))
        crop = img[diffs[0]:diffs[1], diffs[2]:diffs[3]]

        run_ssim, _ = ssim(crop, output, full=True, multichannel=True, channel_axis=2)
        run_psnr = psnr(crop, output)
        run_mse = mse(crop, output)
        metrics.write(orig + ' | SSIM: ' + str(run_ssim) + ' | PSNR: ' + str(run_psnr) + '\n')
        running_ssim += run_ssim
        running_psnr += run_psnr
        running_mse += run_mse
    metrics.write('Real-ESRGAN | Average SSIM: ' + str(running_ssim/len(picks)) + ' | PSNR: ' + str(running_psnr/len(picks)) + '\n\n\n')